# Lösung: Generierung der Exportformate für Lingo und Solr aus dem Bibsonomy-Suchergebnis

In [1]:
import json
import requests
from langdetect import detect

url = "https://www.bibsonomy.org/json/search/Bibliothek?items=1000&duplicates=merged"

data = requests.get(url).json() 

items = data['items']
publications = [item for item in items if item['type'] == 'Publication']
german_pubs = [item for item in publications if detect(item['label'].strip()) == 'de']

## Generieren des Lingo-Exportformats

Das Format für Lingo sieht so aus:

```
[id.]
text.

[id.]
text.

<...>
```

Die ID muss numerisch sein, daher ist es am einfachsten, diese für jeden Datensatz hochzuzählen.

Als Text verwenden wir die beiden textuellen Felder 'label' (=Titel), und, wenn vorhanden, 'abstract'.

In [2]:
with open('output/bibsonomy_to_lingo.txt', 'w') as outfile:
    for id, data in enumerate(german_pubs):
        outfile.write('['+str(id)+'.]\n')
        outfile.write(data['label']+'\n')
        if 'abstract' in data:
            outfile.write(data['abstract']+'\n')
        outfile.write('\n')

## Generieren des Solr-Exportformats

Solr akzeptiert bereits JSON als Importformat, daher könnten wir auch beinahe den Datensatz wie er jetzt ist übergeben. Selbst eine id ist darin bereits vorhanden (es handelt sich um die eindeutige bibsonomy URL zum Datensatz).

Wir wollen allerdings die Daten vorher noch etwas bereinigen, insbesondere unnötige Felder weglassen und Felder mit schlechten Bezeichnungen umbenennen ('label' -> 'title').

In [3]:
for item in german_pubs:
    item['title'] = item.pop('label')
    item.pop('intraHash', None)
    item.pop('interHash', None)
    item.pop('user', None)
    item.pop('dnbtitleid', None)
    item.pop('bibtexKey', None)    

In [4]:
print(json.dumps(german_pubs, indent=4))

[
    {
        "type": "Publication",
        "id": "https://www.bibsonomy.org/bibtex/2fd7893b3cca89352da8b487884c61cc6/genealogie",
        "tags": [
            "dnb",
            "merged:genealogie"
        ],
        "description": "",
        "date": "2015-09-08 22:57:25",
        "changeDate": "2015-09-08 22:57:25",
        "count": 1,
        "pub-type": "phdthesis",
        "year": "1929",
        "url": "",
        "author": [
            "Heinrich Stute"
        ],
        "authors": [
            {
                "first": "Heinrich",
                "last": "Stute"
            }
        ],
        "title": "Studien \u00fcber den Gebrauch der Instrumente in dem italienischen Kirchen-Orchester des 18. Jahrhunderts: Ein Beitr. z. Gesch. d. instrumental begleit. Messe in Italien ; (Auf Grund des Materials in d. Santini-Bibliothek zu M\u00fcnster i. W.)"
    },
    {
        "type": "Publication",
        "id": "https://www.bibsonomy.org/bibtex/24944df9217116b21496436999c1aa13a

In [5]:
with open('output/bibsonomy_to_solr.json', 'w') as outfile:
    json.dump(german_pubs, outfile, indent=4)